# Project: LSTM_module_predict

학습된 모델로 음성파일 스팸 예측

## 순서

1. import: 필요한 모듈 import
2. 파일 불러오기: 모델 학습에 사용된 객체, 변수 등 불러오기
3. STT: 음성파일 STT변환
4. 전처리: 텍스트 토큰화
5. 예측: 스팸 탐지

## 모듈 import

In [1]:
import sys
import json
import pickle
import numpy as np

import speech_recognition as sr

import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request

## GPU 셋팅

GPU 사용 시 활성화

In [63]:
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

## 파일 이름 선언

필요 변수, 모델, 토큰 객체 등 파일 이름 선언

In [2]:
file_name = "LSTM_module_ver4.0"

## 파일 불러오기

tokenizer 객체 불러오기

In [3]:
with open(file_name+'_tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)

변수 불러오기

In [4]:
with open(file_name+'_variable.json') as f:
    var = json.load(f)

불용어 단어 불러오기

In [5]:
with open(file_name+'_stopwords.json') as f:
    stopwords = json.load(f)

불러온 변수 초기화

In [6]:
min_data = var['min_data'] # 데이터 문자열 최소 길이
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)
trunc_type = var['trunc_type']
padding_type = var['padding_type']

print('min_data:',min_data)
print('max_len:',max_len)
print('trunc_type:',trunc_type)
print('padding_type:',padding_type)

min_data: 5
max_len: 40
trunc_type: post
padding_type: post


## STT 변환

음성파일 STT변환
* `language`: 언어 설정
* 약 80~100MB 용량 제한

In [69]:
r = sr.Recognizer()
harvard = sr.AudioFile('spam1.flac') # 100MB 용량 제한
with harvard as source:
    audio = r.record(source)

t = r.recognize_google(audio, language='ko-KR')
print(t)

여보세요 예 안녕하세요 서울중앙지검 유재민 수사관입니다 본인 되시나요 지금 본인 명의 도용 사건에 연루돼서 가지고 혹시 알고 계셨나요 혹시 본인 39세 남성의 황재연을 아시나요 경북 창원에 거주 중이고요 모르겠는데요 그럼 혹시 창원 방문하신 적 있으신가요 최근에 최근 6개월 이내 창원 방문하신 적 있으신가요가 본 적이 없는데요 그러시고 그럼 본인 최근에 모카 전화 통장분실 된 적 있으신가요 적 없어요 없으시고 지금 본인 김유정 씨께서 개인정보유출이 돼 가지고 이제 황재연 왜 13명 금융범죄 사기단이 은병 취득을 목적으로 대포통장을 개설해 가지고 지금 하기를 치고 있다가 이제 불법으로 검거를 하였습니다 저희가 아니 이제 개인정보 어떻게 일주일 제가 지갑을 잃어버린 적도 없고 그거는 아마 뭐 뭐 어쩌다가 뭐 무슨 홈페이지 같은데 들어가다가 유출된 걸 수도 있고 방법은 여러가지가 많거든요네 그래서 제 개인 정보가 이용되고 있다고요 그래 가지고 지금 본인 명의로 통장 개설 돼 가지고 지금 그런 생에 어디 은행 통장 2개월 된 거죠 신한은행입니다 신한은행 신한은행 통장 개설 없으시죠 없습니다 그러면 이게 저희가 지금 수사를 진행중이기 때문에 이제 본인께서 협조를 해 주셔야 되는데 혹시 통화 괜찮으신가요 아 지금 회사 긴 한데 내 통화할 수 있습니다네 본인 일단은 그러면 이제 녹취 수사를 진행할 예정인데 녹취 녹취 녹음을 해 가지고 아무래도 저희가 지금 이사 중이다 보니까 녹음을 해야 되는 상황이 나서 진행을 해야 될 거 같은데 본인 동의하시나요 이거보다 바쁜데 이용 되는 거 아닌가요 유정 씨 저 서울 중앙지검입니다 지금 경찰한테 이렇게 나오시면 안 되죠 아니 제가 어떡해 믿을 수가 있죠 경찰 경찰이니까 이제 서울 중앙지검 이겠죠 본인 지금 못 믿으시나요 못 믿으시면 저희가 소환장을 발고 해 줄 수도 있어요 직접 그러면 서울중앙지검 가지고 오셔야 됩니다네 그럼 내일 보내 주세요 제가 직접가 보겠습니다 아 그렇게 하신다고네 그렇게 할게요 예 알겠습니다네


In [37]:
t = "요새 허리가 쑤시다 어떡하냐 며늘아"

## 전처리: 토큰화 및 불용어 처리

Okt 객체 선언

In [38]:
okt = Okt()

토큰화 및 불용어 처리

In [39]:
def preTreatment(data):
    global stopwords
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp

data = preTreatment(t)

## 모델 예측

학습된 모델 불러오기

In [40]:
model = tf.keras.models.load_model(file_name+'.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 16)            284736    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 305,537
Trainable params: 305,537
Non-trainable params: 0
_________________________________________________________________


정수 인코딩 후 모델 예측

In [41]:
def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(data, maxlen = max_len, truncating = trunc_type, padding = padding_type)
    score = float(model.predict(pad_new))
    return score

score_result = sentiment_predict(data)

## 예측 확률 출력

In [42]:
print("{:.2f}% 확률로 스팸입니다.\n".format(score_result * 100))

0.60% 확률로 스팸입니다.

